<a href="https://colab.research.google.com/github/taoyafan/Master_Graduation_Project/blob/master/make_data_file_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

教程参考：

[想免费用谷歌资源训练神经网络？Colab 详细使用教程](https://jinkey.ai/post/tech/xiang-mian-fei-yong-gu-ge-zi-yuan-xun-lian-shen-jing-wang-luo-colab-xiang-xi-shi-yong-jiao-cheng#toc_2)

[Google Colab Free GPU Tutorial](https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d)

# 授权登录、挂载云盘

In [8]:
# !pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive

drive.mount('/content/drive/')
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# 将数据从云盘读入虚拟机根目录

## 挂载云盘后直接使用 cp 命令复制文件

In [27]:
file_name = "bytecup.corpus.validation_set.txt"
# file_name = "for_test.txt"
# file_name = "bytecup.corpus.train.0.txt"
file_dir = "drive/My\ Drive/code"

src_file = os.path.join(file_dir, 'for_test.txt')
os.system("cp %s %s"% (src_file, file_name))

0

## 原始方法，不需要挂载云盘

显示云盘根目录的文件

In [9]:
# file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
# for file1 in file_list:
#       print('title: %s, id: %s, mimeType: %s' % (file1['title'], file1['id'], file1["mimeType"]))


title: code, id: 18ijhbBiNtaqpgiNwzxAf6a3W13Pc8qSa, mimeType: application/vnd.google-apps.folder
title: Colab Notebooks, id: 1AY8s8p79p0Isoh3Za7zPf3TgLnLanjmp, mimeType: application/vnd.google-apps.folder


打开 code 文件

In [10]:
# file_list = drive.ListFile({'q': "'18ijhbBiNtaqpgiNwzxAf6a3W13Pc8qSa' in parents and trashed=false"}).GetList()
# for file1 in file_list:
#       print('title: %s, id: %s, mimeType: %s' % (file1['title'], file1['id'], file1["mimeType"]))
  

title: bytecup.corpus.train.0.txt, id: 18X4DEun0b3owB26YdHaPF5nzHO7j-a0N, mimeType: text/plain
title: bytecup.corpus.validation_set.txt, id: 1N6UdW9cEm4lxcR_lW0PEDgmdWG1IVoAb, mimeType: text/plain
title: for_test.txt, id: 1X546mxQmuWp7gpA9mzzHh3PO-nvJzz0l, mimeType: text/plain
title: get_to_the_point, id: 1nzipK056ECQjHX8-mnKLpltqhT5P4xeU, mimeType: application/vnd.google-apps.folder


读取云盘中的文件内容

In [11]:
def get_file_content(file_name, show = False):
    id = [file['id'] for file in file_list if file['title'] == file_name][0]
  
    if id == []:
        print("There are not a file named %s"% file_name)
        return []
  
    print("Matched file named %s, and the id is %s"% (file_name, id))
    file = drive.CreateFile({'id': id})
    txt = file.GetContentString()
    if show:
        print("This is the content of file %s :\n\n"% file_name)
        print(txt)
    
    print("Read file complete, length of file is %d" % len(txt))
    return txt
  
# txt = get_file_content(file_name, show = False)

Matched file named for_test.txt, and the id is 1X546mxQmuWp7gpA9mzzHh3PO-nvJzz0l
Read file complete, length of file is 19331


内容写入虚拟机

将内容写入根目录


In [12]:
# import os
# with open(file_name, 'w') as f:
#     f.write(txt)
# !ls

adc.json  drive  for_test.txt  sample_data


读取根目录文件，测试写入成功

In [0]:
def show_file(str):
    with open(str, 'r') as load_f:
        print("This is the content of file %s :\n\n"% str)
        lines = load_f.readlines()
        for i, line in enumerate(lines):
            print(line)
        print("\n\n")
    
# show_file(file_name)

删除根目录文件（如果文件无用）

In [0]:
# os.remove(file_name)
# !ls

# 开始对数据进行预处理

## 文件路径设置

In [0]:
import time
import os

stories_dir = ""
split_stories_dir = "split_stories"
tokenized_stories_dir = "tokenized_stories"
finished_files_dir = "finished_files"
chunks_dir = os.path.join(finished_files_dir, "chunked")
# file_names = ["bytecup.corpus.validation_set.txt"]
file_names = ["for_test.txt"]
# file_names = ["bytecup.corpus.train.{}.txt".format(i) for i in range(8,9)]

if not os.path.exists(split_stories_dir): os.makedirs(split_stories_dir)
if not os.path.exists(tokenized_stories_dir): os.makedirs(tokenized_stories_dir)
if not os.path.exists(finished_files_dir): os.makedirs(finished_files_dir)
if not os.path.exists(chunks_dir): os.makedirs(chunks_dir)


## 将每个数据集文件拆分个多个小文件
每个小文件包含一个故事的内容和标题，以 ID 命名

In [4]:
import json
import os

for name in file_names:
    file_dir = os.path.join(stories_dir, name)
    heads = []
    desc = []

    print('Opening file: %s...' % name)
    with open(file_dir, 'r') as load_f:
        lines = load_f.readlines()
        for i, line in enumerate(lines):
            txt_dic = json.loads(line)

            split_file_dir = os.path.join(split_stories_dir, str(txt_dic['id']))
            with open(split_file_dir, "w") as f:
                f.write(txt_dic['content'].replace('.', '. '))
                f.write(' . ')
                if('title' in txt_dic.keys()):
                    f.write("\n\n@title.\n")
                    f.write(txt_dic['title'])
                
    print("completed\n")

Opening file: for_test.txt...
completed



## 对每篇文章进行分词（分句）

### 安装 java

In [12]:
!apt-get install openjdk-8-jdk
!java -version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jdk is already the newest version (8u181-b13-1ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
openjdk version "1.8.0_181"
OpenJDK Runtime Environment (build 1.8.0_181-8u181-b13-1ubuntu0.18.04.1-b13)
OpenJDK 64-Bit Server VM (build 25.181-b13, mixed mode)


### 选择是否分句
若为 DIVIDE_SENTENCE = True 则分句加分词，否则只分词

In [0]:
DIVIDE_SENTENCE = False

### 只分词不分句
先生成映射文件 mapping，再使用 PTBTokenizer 对文件进行直接操作

In [34]:

import subprocess

if not DIVIDE_SENTENCE:

    time_start = time.time()
    
    stories = os.listdir(split_stories_dir)
    #生成映射文件 mapping.txt
    print("Making list of files to tokenize...")
    with open("mapping.txt", "w") as f:
        for s in stories:
            f.write("%s \t %s\n" % (os.path.join(split_stories_dir, s), os.path.join(tokenized_stories_dir, s)))
    print("completed\n")
    
    # Doesn't work
    # command = ['java', 'edu.stanford.nlp.process.DocumentPreprocessor', '-ioFileList', '-preserveLines', 'mapping.txt']
    
    command = ['java', 'edu.stanford.nlp.process.PTBTokenizer', '-ioFileList', '-preserveLines', 'mapping.txt']
    subprocess.call(command)
    
    # Doesn't work
    # os.system("java edu.stanford.nlp.process.DocumentPreprocessor -ioFileList -preserveLines mapping.txt")
    
    time_end = time.time()
    print('Consume time:', time_end - time_start)
    os.remove("mapping.txt")

Making list of files to tokenize...
completed

Consume time: 0.08802556991577148


### 分句加分词
无法利用映射文件进行分词，有BUG，故只能一个一个文件分词

In [0]:
import os
if DIVIDE_SENTENCE:
    
    time_start = time.time()
    
    stories = os.listdir(split_stories_dir)
    for s in stories:
        os.system("java edu.stanford.nlp.process.DocumentPreprocessor -preserveLines < %s > %s" %
                  (os.path.join(split_stories_dir, s), os.path.join(tokenized_stories_dir, s)))

    time_end = time.time()
    print('Consume time:', time_end - time_start)

## 格式化文章并存储

### 根据文件地址读文件，返回行列表

In [0]:
def read_text_file(text_file):
    lines = []
    with open(text_file, "r") as f:
        for line in f:
            lines.append(line.strip())
        return lines

### 根据文件内容获得文章和摘要

In [0]:
SENTENCE_START = '<s>'
SENTENCE_END = '</s>'

def get_art_abs(story_file):
    lines = read_text_file(story_file)

    # Lowercase everything
    lines = [line.lower() for line in lines]

    # Separate out article and abstract sentences
    article_lines = []
    titles = []
    next_is_title = False
    for idx,line in enumerate(lines):
        if line == "":
            continue # empty line
        elif line.startswith("@title"):
            next_is_title = True
        elif next_is_title:
            titles.append(line)
        else:
            article_lines.append(line)

    # Make article into a single string
    article = ' '.join(article_lines)

    # Make abstract into a signle string, putting <s> and </s> tags around the sentences
    title = ' '.join(["%s %s %s" % (SENTENCE_START, sent, SENTENCE_END) for sent in titles])

    return article, title

### 写入 bin 文件

In [0]:
from tensorflow.core.example import example_pb2
import struct

def write_to_bin(src_dir, out_file):
    
    with open(out_file, 'wb') as writer:
        
        stories = os.listdir(src_dir)
        for idx, s in enumerate(stories):
            story_file = os.path.join(tokenized_stories_dir, s)
            article, title = get_art_abs(story_file)
        #     print("article: \n%s\n\n"%article)
        #     print("title: \n%s\n" % title)

            tf_example = example_pb2.Example()
            tf_example.features.feature['article'].bytes_list.value.extend([article.encode()])
            tf_example.features.feature['abstract'].bytes_list.value.extend([title.encode()])
            tf_example_str = tf_example.SerializeToString()

            str_len = len(tf_example_str)
            writer.write(struct.pack('q', str_len))
            writer.write(struct.pack('%ds' % str_len, tf_example_str))
            
write_to_bin(tokenized_stories_dir, os.path.join(finished_files_dir, "test.bin"))

### 将文件分块，每个块的文章个数为 CHUNK_SIZE

In [41]:
CHUNK_SIZE = 1000 # num examples per chunk, for the chunked data

def chunk_file(set_name):
    in_file = os.path.join(finished_files_dir, '%s.bin' % set_name)
    with open(in_file, 'rb') as reader:
        chunk = 0
        finished = False
        while not finished:
            chunk_fname = os.path.join(chunks_dir, '%s_%03d.bin' % (set_name, chunk)) # new chunk
            with open(chunk_fname, 'wb') as writer:
                for _ in range(CHUNK_SIZE):
                    len_bytes = reader.read(8)
                    if not len_bytes:
                        finished = True
                        break
                    str_len = struct.unpack('q', len_bytes)[0]
                    example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
                    writer.write(struct.pack('q', str_len))
                    writer.write(struct.pack('%ds' % str_len, example_str))
                chunk += 1
            
def chunk_all():
    # Chunk the data
#     for set_name in ['train', 'val', 'test']:
    for set_name in ['test']:
        print("Splitting %s data into chunks..." % set_name)
        chunk_file(set_name)
    print("Saved chunked data in %s" % chunks_dir)
    
# 分块
chunk_all()

Splitting test data into chunks...
Saved chunked data in finished_files/chunked


In [42]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Running setup.py bdist_wheel for gputil ... - done
  Stored in directory: /root/.cache/pip/wheels/17/0f/04/b79c006972335e35472c0b835ed52bfc0815258d409f560108
Successfully built gputil
  Running setup.py bdist_wheel for humanize ... - done
  Stored in directory: /root/.cache/pip/wheels/69/86/6c/f8b8593bc273ec4b0c653d3827f7482bb2001a2781a73b7f44
Successfully built humanize
Gen RAM Free: 12.7 GB  | Proc size: 296.9 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB
